# [Enums and Pattern Matching](https://doc.rust-lang.org/book/ch06-00-enums.html)
Time to explore the world of `enum` in Rust!

## [Defining an Enum](https://doc.rust-lang.org/book/ch06-01-defining-an-enum.html#defining-an-enum)
Nothing too surprising about the syntax:
```rust
enum IpAddrKind {
    V4,
    V6,
}
```
Though the introduction is quite interesting, implying they may have more responsibilities than in other languages.

They seem to be talking about use as an interface type?

```rust
enum IpAddrKind {
    V4,
    V6,
}
...
    let four = IpAddrKind::V4;
    let six = IpAddrKind::V6;
```

So this is working similar to a namespace?

That raises a lot of questions.

```rust
fn route(ip_kind: IpAddrKind) {}
```

Well that confirms their use in inheritance.

The syntax of Rust is quite strongly tied to the base types.

Another use here is interesting:
```rust
    enum IpAddrKind {
        V4,
        V6,
    }

    struct IpAddr {
        kind: IpAddrKind,
        address: String,
    }

    let home = IpAddr {
        kind: IpAddrKind::V4,
        address: String::from("127.0.0.1"),
    };

    let loopback = IpAddr {
        kind: IpAddrKind::V6,
        address: String::from("::1"),
    };
```
It's... constructing properties in a struct?

I presume this implies the required compiler validation.

And you can just create a tuple struct within it:

```rust
    enum IpAddr {
        V4(u8, u8, u8, u8),
        V6(String),
    }
```

Wow.

And then you can easily create internal structures within the elements:
```rust
enum Message {
    Quit,
    Move { x: i32, y: i32 },
    Write(String),
    ChangeColor(i32, i32, i32),
}
```

I've gotta hand it to them, this is a nice type system.

Then you can define methods by extending the base enum:
```rust
    impl Message {
        fn call(&self) {
            // method body would be defined here
        }
    }

    let m = Message::Write(String::from("hello"));
    m.call();
```

So I guess you'd use another `impl` block for each of the class options mentioned?

What if you don't? Do they get the "default" implementation from the base class?

How do you create an abstract method that child classes are required to implement?









## [The Option Enum and Its Advantages Over Null Values](https://doc.rust-lang.org/book/ch06-01-defining-an-enum.html#the-option-enum-and-its-advantages-over-null-values)
The tutorial talks a little bit about `null` and the problems it causes. 

Then they go on to say Rust **doesn't have a null type!**

Given my strong dislike of the use of null in so many languages I'm very happy to hear that.

I've seen this type abused so much:
 * For a search with no results... just use an empty list.
 * For error conditions... use an exception.
 * For an invalid result
 * As a default input argument for optional values.
    * This might be the one I agree with, if an abstract type isn't possible.

I've used programming languages with non-nullable types or having a specific null-type.

But I don't think I've ever seen null removed entirely.

Well... it sounds good but what to they do instead for blank or missing values?

So they say there's an option enum present in the default scope:
```rust
enum Option<T> {
    None,
    Some(T),
}
    let some_number = Some(5);
    let some_char = Some('e');

    let absent_number: Option<i32> = None;
```

It allows a value or an absent one... 

So this is a specific nullable type not like NoneType.

Hmm...  

```rust
    let x: i8 = 5;
    let y: Option<i8> = Some(5);

    let sum = x + y;
```
This code fails to compile because is `Option<i8>` is a now a separate (nullable) type.

So it follows that no regular variable can contain null. 

`Option<T>` has a large number of methods built in for handling a possible null value:
 * Allowing a default value to be specified.
 * Using a method to retrieve an alternative value.
 * Raising a panic if null (discouraged).
 * Querying to check if null.
 
To sum it up:
 * All types are non-nullable by default.
 * The `Option` type allowing nulling.
     * It create a separate type to prevent mixing with non-nullable types.
     * If forces you to create handlers specifically to deal with nullable items.
     * It defines a lot of different handling methods when a null does occur.
 * I'm optimisic about this system.
 

Enough notetaking, I need to write some code to test this out. 











I rewrote my shape program from the previous lesson to be a lot nicer.
```rust
use std::f64::consts::PI;

fn main() {
    let shape = Shape::Circle { radius: 2.0 };
    shape.print_properties();
    let shape = Shape::Square { side_length: 3.0 };
    shape.print_properties();
    let shape = Shape::Rectangle { width: 3.0, height: 1.5 };
    shape.print_properties();
    let shape = Shape::EquilateralTriangle { side_length: 4.0 };
    shape.print_properties();
    let shape = Shape::ScaleneTriangle { a: 2.0, b: 3.0, c: 4.0 };
    shape.print_properties();
}


enum Shape {
    Circle { radius: f64 },
    Square { side_length: f64 },
    Rectangle { width: f64, height: f64 },
    EquilateralTriangle { side_length: f64 },
    ScaleneTriangle { a: f64, b: f64, c: f64 },
}

impl Shape {
    fn get_area(&self) -> f64 {
        match self {
            Shape::Circle { radius } => PI * radius.powi(2),
            Shape::Square { side_length } => side_length.powi(2),
            Shape::Rectangle { width, height } => width * height,
            Shape::EquilateralTriangle { side_length } => 
                (f64::sqrt(3.0) / 4.0) * side_length.powi(2),
            Shape::ScaleneTriangle { a, b, c } => {
                let s = (a + b + c) / 2.0;
                let product = s * (s - a) * (s - b) * (s - c);
                product.sqrt()
            },
        }
    }
    
    fn get_properties_as_text(&self) -> String {
        match self {
            Shape::Circle { radius } => 
                format!("Type: Circle, Radius: {radius}"),
            Shape::Square { side_length } => 
                format!("Type: Square, Length: {side_length}"),
            Shape::Rectangle { width, height } => 
                format!("Type: Rectangle, Width: {width}, Height: {height}"),
            Shape::EquilateralTriangle { side_length } =>
                format!("Type: EquilateralTriangle, Side Length: {side_length}"),
            Shape::ScaleneTriangle { a, b, c } =>
                format!("Type: ScaleneTriangle, Side A: {a}, Side B: {b}, Side C: {c}"),
        }
    }
    
    fn print_properties(&self) {
        println!("-----------------------------");
        println!("Shape Info");
        println!("{}", self.get_properties_as_text());
        println!("Area: {}", self.get_area());
    }
    
}
```

Some thoughts:
 * The shapes object is more procedural than OOP.
     * Equally valid I suppose, but it leads to long complicated methods.
         * This is why people hate switch statements...
     * My original plan was to make a lot of struct object and put them into an enum.
     * This failed when I couldn't figure out the constructor for a struc inside and enum.
     * Instead I went back to the arms match we learnt earlier with `match self`.
        * Love the fact it gives an error if you miss a possible option.
     * Is there a `.square()` method? It seems a bit weird to be using `.powi(2)` for that.
     * `format!()` seems to do the same formatting what `println!()` does without printing.
     


## [The match Control Flow Construct](https://doc.rust-lang.org/book/ch06-02-match.html)
I've already done plenty on `match` in this tutorial, but it seems like there's more features to learn!

They give the example of a coin sorting machine.
```rust
enum Coin {
    Penny,
    Nickel,
    Dime,
    Quarter,
}

fn value_in_cents(coin: Coin) -> u8 {
    match coin {
        Coin::Penny => 1,
        Coin::Nickel => 5,
        Coin::Dime => 10,
        Coin::Quarter => 25,
    }
}
```

Well, I already learnt that syntax for the last program.

It more of the same `Enum::Option => expression` syntax.

It's a nice way to do switch statements

However, I've concerned about the amount of code that could potential be introduced with these blocks.

It can easily lead to excessively long functions.

Perhaps it would be better to point to a function for any non-trivial expression?

## [Matching with Option<T>](https://doc.rust-lang.org/book/ch06-02-match.html)
Well this seems like a more interesting case.

```rust
    fn plus_one(x: Option<i32>) -> Option<i32> {
        match x {
            None => None,
            Some(i) => Some(i + 1),
        }
    }

    let five = Some(5);
    let six = plus_one(five);
    let none = plus_one(None);
```
The combination of `match` and `Option` allows us to define different code paths for null parameters.

At first I thought there might be four possible paths, but this is actually a unary increment.

What about with binary operations where there are two input variables?

Do you use a sub-match?

Again, it's nice that you can't forgot null handling or it won't compile.

So here's the catch-all that some languages call `default`:
```rust
    match dice_roll {
        3 => add_fancy_hat(),
        7 => remove_fancy_hat(),
        other => move_player(other),
    }
```
You can use `other` if the value is needed.

Or alternatively just an underscore if it isn't.

```rust
        _ => reroll(),
```

I suppose the underscore is necessary to avoid the compiler complaining if the value is not used.

They also define:
```rust
        _ => (),
```
If no code is to be run, because *this case must be explicitly declared*.



## [Concise Control Flow with if let](https://doc.rust-lang.org/book/ch06-03-if-let.html)
What... is that?


So it seems like this is a special case of `match` where code only executes on one specific value.

It also seems to be checking the pattern is not `None`.

The example given is this code:
```rust
    let config_max = Some(3u8);
    match config_max {
        Some(max) => println!("The maximum is configured to be {max}"),
        _ => (),
    }
```

Turning into this:
```rust
    let config_max = Some(3u8);
    if let Some(max) = config_max {
        println!("The maximum is configured to be {max}");
    }
```

I can see how it might make it a little more consise, but I'd probably still prefer to go through the whole list.

An `else` can also be added to `if let` for more flow control.